# Pipeline 

In [1]:
%load_ext autoreload
%autoreload 2

from averitec import Datapoint
from evidence_generation import GptEvidenceGenerator, FixedFewShotEvidenceGenerator, DynamicFewShotEvidenceGenerator
from classification import DefaultClassifier, HuggingfaceClassifier, AverageEnsembleClassifier, LogRegEnsembleClassifier, RandomForestClassifier
from retrieval import SimpleFaissRetriever
from pipeline import Pipeline, MockPipeline
import pickle
from labels import label2id, id2label
import numpy as np
from sklearn.metrics import classification_report
import random
from tqdm import tqdm
random.seed(111)

import json

## Pipeline test

In [5]:
with open('/mnt/data/factcheck/averitec-data/data/dev.json') as f:
    dataset = json.load(f)
    claim_id = 0
    for el in dataset:
        el['claim_id'] = claim_id
        claim_id += 1

In [6]:
datapoint = Datapoint.from_dict(dataset[245])
datapoint

Datapoint(claim='Potential voter fraud in the 2020 U.S Presidential Election is caused by mail-in ballots being sent to a mans  deceased parents in Mercer County in New Jersey.', claim_id=245, claim_date='29-9-2020', speaker='John P Papamanolis', original_claim_url='https://web.archive.org/web/20201030014600/https://perma.cc/E4X3-JXLJ?type=image', reporting_source='Facebook', location_ISO_code='US', label='Refuted', metadata={'required_reannotation': True, 'justification': 'The claim is badly phrased and suggests that all potential voter fraud was caused by the ballots sent in Mercer County. ', 'fact_checking_article': 'https://web.archive.org/web/20201030014600/https://factcheck.afp.com/mail-ballot-errors-spark-voter-fraud-claims-us', 'claim_types': ['Causal Claim'], 'fact_checking_strategies': ['Written Evidence', 'Consultation'], 'questions': [{'question': 'Are there signs of voter fraud in US elections?', 'answers': [{'answer': 'Social media users, including one of US President Don

In [7]:
retriever = SimpleFaissRetriever(k=5, path="/mnt/data/factcheck/averitec-data/data_store/vecstore/dev/6k")
retrieval_result = retriever(datapoint)
retrieval_result

RetrievalResult(documents=[Document(metadata={'url': 'https://observers.france24.com/en/20201014-voter-fraud-us-voters-post-photos-mail-inl-ballots-trump', 'context_before': "'Voter Fraud Alert!' - US voters post photos of their unsolicited mail-in ballots online This year’s US presidential election will see unprecedented numbers of Americans sending in their ballots by mail due to the coronavirus pandemic. However, as states moved to expand voting by mail, President Trump and his supporters have cast doubt on the integrity of mail-in ballots, particularly “universal mail-in voting” where states automatically send ballots to all registered and active voters. In recent weeks, voters have been expressing concern on social media after receiving mail-in ballots for people who have moved or passed away. On September 30, Twitter user Richie McGinniss posted a picture of what he said were four ballots from Washington D.C. for people who no longer lived in his home. His post provoked a flurry 

In [8]:
evidence_generator = GptEvidenceGenerator("gpt-4o")
evidence_generation_result = evidence_generator(datapoint, retrieval_result)
evidence_generation_result

EvidenceGenerationResult(evidences=[Evidence(question='Were mail-in ballots sent to deceased individuals in New Jersey during the 2020 U.S. Presidential Election?', answer='Yes, there were reports of mail-in ballots being sent to deceased individuals in New Jersey.', url='https://observers.france24.com/en/20201014-voter-fraud-us-voters-post-photos-mail-inl-ballots-trump', scraped_text="'Voter Fraud Alert!' - US voters post photos of their unsolicited mail-in ballots online This year’s US presidential election will see unprecedented numbers of Americans sending in their ballots by mail due to the coronavirus pandemic. However, as states moved to expand voting by mail, President Trump and his supporters have cast doubt on the integrity of mail-in ballots, particularly “universal mail-in voting” where states automatically send ballots to all registered and active voters. In recent weeks, voters have been expressing concern on social media after receiving mail-in ballots for people who hav

In [9]:
for ev in evidence_generation_result.evidences:
    print(ev)

Evidence(question='Were mail-in ballots sent to deceased individuals in New Jersey during the 2020 U.S. Presidential Election?', answer='Yes, there were reports of mail-in ballots being sent to deceased individuals in New Jersey.', url='https://observers.france24.com/en/20201014-voter-fraud-us-voters-post-photos-mail-inl-ballots-trump', scraped_text="'Voter Fraud Alert!' - US voters post photos of their unsolicited mail-in ballots online This year’s US presidential election will see unprecedented numbers of Americans sending in their ballots by mail due to the coronavirus pandemic. However, as states moved to expand voting by mail, President Trump and his supporters have cast doubt on the integrity of mail-in ballots, particularly “universal mail-in voting” where states automatically send ballots to all registered and active voters. In recent weeks, voters have been expressing concern on social media after receiving mail-in ballots for people who have moved or passed away. On September

In [7]:
evidence_generator2 = FixedFewShotEvidenceGenerator("gpt-4o")
evidence_generation_result2 = evidence_generator2(datapoint, retrieval_result)
evidence_generation_result2.evidences

[Evidence(question='Were mail-in ballots sent to deceased individuals in New Jersey during the 2020 U.S. Presidential Election?', answer='Yes, there were reports of mail-in ballots being sent to deceased individuals in New Jersey.', url='https://observers.france24.com/en/20201014-voter-fraud-us-voters-post-photos-mail-inl-ballots-trump', scraped_text="'Voter Fraud Alert!' - US voters post photos of their unsolicited mail-in ballots online This year’s US presidential election will see unprecedented numbers of Americans sending in their ballots by mail due to the coronavirus pandemic. However, as states moved to expand voting by mail, President Trump and his supporters have cast doubt on the integrity of mail-in ballots, particularly “universal mail-in voting” where states automatically send ballots to all registered and active voters. In recent weeks, voters have been expressing concern on social media after receiving mail-in ballots for people who have moved or passed away. On Septembe

In [20]:
for ev in evidence_generation_result2.evidences:
    print(ev)

Evidence(question='Did Joe Biden explicitly state his support for the Green New Deal during the first 2020 presidential debate?', answer='No, Joe Biden explicitly stated that he does not support the Green New Deal during the first 2020 presidential debate.', url='https://www.washingtonexaminer.com/opinion/1528306/biden-disavows-green-new-deal-co-sponsored-by-kamala-harris/', scraped_text='\nDemocratic nominee Joe Biden may need to have a sit-down with his running mate, Sen. Kamala Harris of California. On Tuesday, during the first (and hopefully last) 2020 presidential debate, Biden stated outright that he does not support the Green New Deal, a preposterous multitrillion-dollar plan to make the economy more environmentally friendly. “The Green New Deal will pay for itself as we move forward,” Biden alleged. “We’re not going to build plants that, in fact, are great polluting plants.” Moderator Chris Wallace interjected for clarity, “But do you support the Green New Deal?” “No,” Biden an

In [10]:
evidence_generator3 = DynamicFewShotEvidenceGenerator("gpt-4o", k=5)
evidence_generation_result3 = evidence_generator3(datapoint, retrieval_result)
evidence_generation_result3

EvidenceGenerationResult(evidences=[Evidence(question='Were mail-in ballots sent to deceased individuals in New Jersey?', answer='Yes, there were reports of mail-in ballots being sent to deceased individuals in New Jersey.', url='https://observers.france24.com/en/20201014-voter-fraud-us-voters-post-photos-mail-inl-ballots-trump', scraped_text="'Voter Fraud Alert!' - US voters post photos of their unsolicited mail-in ballots online This year’s US presidential election will see unprecedented numbers of Americans sending in their ballots by mail due to the coronavirus pandemic. However, as states moved to expand voting by mail, President Trump and his supporters have cast doubt on the integrity of mail-in ballots, particularly “universal mail-in voting” where states automatically send ballots to all registered and active voters. In recent weeks, voters have been expressing concern on social media after receiving mail-in ballots for people who have moved or passed away. On September 30, Tw

In [11]:
for ev in evidence_generation_result3.evidences:
    print(ev)

Evidence(question='Were mail-in ballots sent to deceased individuals in New Jersey?', answer='Yes, there were reports of mail-in ballots being sent to deceased individuals in New Jersey.', url='https://observers.france24.com/en/20201014-voter-fraud-us-voters-post-photos-mail-inl-ballots-trump', scraped_text="'Voter Fraud Alert!' - US voters post photos of their unsolicited mail-in ballots online This year’s US presidential election will see unprecedented numbers of Americans sending in their ballots by mail due to the coronavirus pandemic. However, as states moved to expand voting by mail, President Trump and his supporters have cast doubt on the integrity of mail-in ballots, particularly “universal mail-in voting” where states automatically send ballots to all registered and active voters. In recent weeks, voters have been expressing concern on social media after receiving mail-in ballots for people who have moved or passed away. On September 30, Twitter user Richie McGinniss posted a

In [12]:
evidence_generation_result.metadata["suggested_label"]

array([0.45764028, 0.06193488, 0.45764028, 0.02278457])

In [13]:
datapoint.label

'Refuted'

In [14]:
classifier = DefaultClassifier()
classification_result = classifier(datapoint, evidence_generation_result, retrieval_result)
str(classification_result), classification_result

('Refuted',
 ClassificationResult(probs=array([0.45764028, 0.06193488, 0.45764028, 0.02278457]), metadata=None))

In [17]:
datapoint2 = Datapoint.from_dict(dataset[16])
pipeline = Pipeline(retriever, evidence_generator, classifier)
pipeline_result = pipeline(datapoint2)
pipeline_result

PipelineResult(datapoint=Datapoint(claim='The plentiful evidence we have indicates that masks would not meaningfully help with aerosol transmission of COVID 19.', claim_id=16, claim_date='29-10-2020', speaker='Yinon Weiss', original_claim_url='https://thefederalist.com/2020/10/29/these-12-graphs-show-mask-mandates-do-nothing-to-stop-covid/', reporting_source='The Federalist', location_ISO_code='US', label='Refuted', metadata={'required_reannotation': True, 'justification': 'Evidence finds that surgical face masks could prevent transmission of human coronaviruses and influenza viruses from symptomatic individuals. The CDC and the World Health Organization (WHO), at the time of the claim, also recommended cloth masks for the general public.', 'fact_checking_article': 'https://web.archive.org/web/20210711073635/https://healthfeedback.org/claimreview/wearing-face-masks-helps-reduce-the-spread-of-covid-19-but-it-only-confers-partial-protection-and-should-be-combined-with-additional-public-h

In [18]:
str(pipeline_result.classification_result), datapoint2.label

('Refuted', 'Refuted')

In [19]:
pipeline_result.to_submission()

{'claim_id': 16,
 'claim': 'The plentiful evidence we have indicates that masks would not meaningfully help with aerosol transmission of COVID 19.',
 'evidence': [{'question': 'What is the primary mode of COVID-19 transmission?',
   'answer': 'COVID-19 primarily spreads through respiratory droplets, but aerosol transmission is also possible.',
   'url': 'https://aaqr.org/articles/aaqr-20-06-covid-0275',
   'scraped_text': 'The unprecedented demand for masks has now attracted attention on their filtration efficiency. Furthermore, the widespread use of disposable masks has led to shortage of filter materials and problems with their haphazard disposal. In this study, a testing system that is based on standardized methods has been established and enhanced to reliably measure the particle filtration efficiency (PFE) of masks. Quality control experiments that examine the filtration efficiency of polystyrene latex (PSL) particles that are 0.1 µm in size and sodium chloride (NaCl) particles th

In [12]:
# pickle dump pipeline result
import pickle
with open('data/pipeline_result.pkl', 'wb') as f:
    pickle.dump(pipeline_result, f)

## Whole dev set example
not really whole, but we will show a sampling :)

In [8]:
import json, random, pickle
from tqdm.notebook import tqdm
sample_size = 2
random.seed(111)

In [86]:
retriever = SimpleFaissRetriever(k=5, path="/mnt/data/factcheck/averitec-data/data_store/vecstore/dev/6k")
evidence_generator = GptEvidenceGenerator("gpt-4o")
classifier = DefaultClassifier()
pipeline = Pipeline(retriever, evidence_generator, classifier)

with open("/mnt/data/factcheck/averitec-data/data/dev.json") as f:
    dataset = json.load(f)
    seq = 0
    for d in dataset:
        d["claim_id"] = seq
        seq += 1


sample = random.sample(dataset, sample_size)

with open(f"/mnt/data/factcheck/averitec-data/data/dev{sample_size}.json", "w") as f:
    json.dump(sample, f)
    
datapoints = [Datapoint.from_dict(d) for d in sample]
submission = []
dump = []

for dp in tqdm(datapoints):
    pipeline_result = pipeline(dp)
    submission.append(pipeline_result.to_submission())
    dump.append(pipeline_result)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/submission_dev{sample_size}.json", "w") as f:
        json.dump(submission, f, indent=4)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/pipeline_result_dev{sample_size}.pkl", "wb") as f:
        pickle.dump(dump, f)

  0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
dump[-1].evidence_generation_result.metadata

{'suggested_label': array([0.1469628 , 0.3994863 , 0.3994863 , 0.05406459]),
 'llm_type': 'gpt-4o',
 'llm_output': {'questions': [{'question': 'Did Donald Trump sign executive orders aimed at lowering prescription drug costs?',
    'answer': 'Yes, Donald Trump signed multiple executive orders aimed at lowering the cost of prescription drugs.',
    'source': '0'},
   {'question': 'When did Donald Trump sign these executive orders?',
    'answer': 'Donald Trump signed these executive orders on a Friday, but the exact date is not specified in the sources.',
    'source': '1'},
   {'question': 'What were the main objectives of the executive orders signed by Trump?',
    'answer': 'The executive orders aimed to allow imports of cheaper medicines, direct drugmaker rebates straight to patients, provide insulin and EpiPens at steep discounts to low-income people, and use lower international prices to pay for some Medicare drugs.',
    'source': '1'},
   {'question': 'Did Trump claim that these

### now lets see if our submission fits the eval script

In [15]:
%run src/prediction/evaluate_veracity.py --label_file data/dev{sample_size}.json --prediction_file data_store/submission_dev{sample_size}.json

Question-only score (HU-meteor):             0.7057333276522711
Question-answer score (HU-meteor):           0.36226711909299614
Veracity F1 scores:
 * Supported:                                0.6666666666666666
 * Refuted:                                  0.0
 * Not Enough Evidence:                      0.0
 * Conflicting Evidence/Cherrypicking:       0.0
 * macro:                                    0.16666666666666666
 * acc:                                      0.5
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.5
 * Veracity scores (meteor @ 0.2):           0.5
 * Veracity scores (meteor @ 0.25):          0.5
 * Veracity scores (meteor @ 0.3):           0.5
 * Veracity scores (meteor @ 0.4):           0.0
 * Veracity scores (meteor @ 0.5):           0.0
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Position Statement):     0.36226711909299614
 * Veracity scores (Quote Verification):     0.32760357724321804
 * Veracity

/home/ullriher/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/ullriher/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1517: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


## Load Pickles

In [20]:
# DUMP_PATH = "/mnt/data/factcheck/averitec-data/data_store/pipeline_result_dev.pkl"
DUMP_PATH = "/mnt/data/factcheck/averitec-data/data_store/submissions/dev_mmr+gpt4o-dfewshot.pkl"
DUMP_PATH_CLAUDE = "/mnt/data/factcheck/averitec-data/data_store/pipeline_result_dev_claude.pkl"
# dump = pickle.load(open(f"/mnt/data/factcheck/averitec-data/data_store/pipeline_result_dev{sample_size}.pkl", "rb"))
dump = pickle.load(open(DUMP_PATH, "rb")) #whole dev set

retrieval_results = [d.retrieval_result for d in dump]
evidence_generation_results = [d.evidence_generation_result for d in dump]
datapoints = [d.datapoint for d in dump]

dump_claude = pickle.load(open(DUMP_PATH_CLAUDE, "rb")) #whole dev set

retrieval_results_claude = [d.retrieval_result for d in dump_claude]
evidence_generation_results_claude = [d.evidence_generation_result for d in dump_claude]
datapoints_claude = [d.datapoint for d in dump_claude]

In [23]:
#split dump in dev_train and dev_test respecting the labels
#divide dump according to the labels
dump_labels = {0: [], 1: [], 2: [], 3: []}

for d in dump:
    dump_labels[label2id[d.datapoint.label]].append(d)

dump_dev_train = []
dump_dev_test = []

for label, data in dump_labels.items():
    split = int(len(data)*0.7)
    dump_dev_train.extend(data[:split])
    dump_dev_test.extend(data[split:])

random.shuffle(dump_dev_train)

retrieval_results_dev_train = [d.retrieval_result for d in dump_dev_train]
evidence_generation_results_dev_train = [d.evidence_generation_result for d in dump_dev_train]
datapoints_dev_train = [d.datapoint for d in dump_dev_train]

retrieval_results_dev_test = [d.retrieval_result for d in dump_dev_test]
evidence_generation_results_dev_test = [d.evidence_generation_result for d in dump_dev_test]
datapoints_dev_test = [d.datapoint for d in dump_dev_test]

In [13]:
complete_results = []

## Classifiers testing

In [24]:
def test_classifier(pipeline_results, datapoints, verbose=False, cls_name=""):
    pipeline_labels = [np.argmax(pr.classification_result.probs) for pr in pipeline_results]
    
    true_labels = [label2id[d.label] for d in datapoints]
    if verbose:
        print("y_pred", np.unique(pipeline_labels, return_counts=True))
        print("y_true", np.unique(true_labels, return_counts=True))

    print(classification_report(true_labels, pipeline_labels, target_names=id2label.values()))
    res_dict = classification_report(true_labels, pipeline_labels, target_names=id2label.values(), output_dict=True)
    res_dict["cls_name"] = cls_name
    return res_dict

In [6]:
def test_pipelines(pipeline, dump, dump_dev_test, cls_name):
    pipeline_results = [pipeline(d.datapoint) for d in dump]
    print(f"Testing {cls_name} on dev")
    res = test_classifier(pipeline_results, [d.datapoint for d in dump], cls_name=f"{cls_name} - dev")
    pipeline_results_dev_test = [pipeline(d.datapoint) for d in dump_dev_test]
    print(f"Testing {cls_name} on dev_test")
    res_dev_test = test_classifier(pipeline_results_dev_test, [d.datapoint for d in dump_dev_test], cls_name=f"{cls_name} - dev_test")
    return res, res_dev_test

### RandomForest classifier (Bryce)

In [25]:
rf_classifier = RandomForestClassifier()

mock_pipeline = MockPipeline(
    classifier=rf_classifier,
    dumps=DUMP_PATH,
)

pipeline_result = mock_pipeline(dump_dev_test[0].datapoint)
pipeline_result.classification_result, str(pipeline_result.classification_result), str(pipeline_result.evidence_generation_result.evidences[0])

(ClassificationResult(probs=array([0.7238869 , 0.08580357, 0.15263095, 0.03767857]), metadata={'predicted_label': 'Refuted'}),
 'Refuted',
 'Evidence(question=\'Was Reed Hastings, the CEO of Netflix, arrested by the FBI?\', answer=\'No, Reed Hastings was not arrested by the FBI.\', url=\'https://www.boomlive.in/world/posts-claiming-netflix-ceo-was-arrested-for-child-pornography-are-false-9919\', scraped_text=\'\\nPosts Claiming Netflix CEO Was Arrested For Child Pornography Are False Law enforcement officials said that there was no arrest and the rumour was published by a fringe news site, where it has since been removed. "The CEO of a Netflix – Reed Hastings – has been caught by FBI agents with 13,000 files of child pornography. He was arrested at his home in California," reads a Facebook post, archived here, shared more than 400 times since it was published on September 12, 2020. Screenshot taken on September 22, 2020, showing the false claim on Facebook The same claim also circulate

In [26]:
pipeline_results = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.76      0.40      0.52        92
                         Supported       1.00      0.05      0.10        37
               Not Enough Evidence       0.15      0.91      0.26        11
Conflicting Evidence/Cherrypicking       0.17      0.50      0.26        12

                          accuracy                           0.36       152
                         macro avg       0.52      0.47      0.29       152
                      weighted avg       0.72      0.36      0.38       152



In [27]:
pipeline_results = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.70      0.47      0.56       305
                         Supported       1.00      0.16      0.27       122
               Not Enough Evidence       0.15      0.74      0.25        35
Conflicting Evidence/Cherrypicking       0.14      0.37      0.20        38

                          accuracy                           0.41       500
                         macro avg       0.50      0.43      0.32       500
                      weighted avg       0.69      0.41      0.44       500



### GPT4o Likert scale based classifier

In [28]:
#GPTClassifier example
gpt_likert_classifier = DefaultClassifier()

mock_pipeline = MockPipeline(
    classifier=gpt_likert_classifier,
    dumps=DUMP_PATH,
)

pipeline_result = mock_pipeline(dump_dev_test[0].datapoint)
pipeline_result.classification_result, str(pipeline_result.classification_result), str(pipeline_result.evidence_generation_result.evidences[0])

(ClassificationResult(probs=array([0.94791499, 0.01736167, 0.01736167, 0.01736167]), metadata=None),
 'Refuted',
 'Evidence(question=\'Was Reed Hastings, the CEO of Netflix, arrested by the FBI?\', answer=\'No, Reed Hastings was not arrested by the FBI.\', url=\'https://www.boomlive.in/world/posts-claiming-netflix-ceo-was-arrested-for-child-pornography-are-false-9919\', scraped_text=\'\\nPosts Claiming Netflix CEO Was Arrested For Child Pornography Are False Law enforcement officials said that there was no arrest and the rumour was published by a fringe news site, where it has since been removed. "The CEO of a Netflix – Reed Hastings – has been caught by FBI agents with 13,000 files of child pornography. He was arrested at his home in California," reads a Facebook post, archived here, shared more than 400 times since it was published on September 12, 2020. Screenshot taken on September 22, 2020, showing the false claim on Facebook The same claim also circulated on social media in Spani

In [14]:
#final evaluation
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "GPT4o Likert")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing GPT4o Likert on dev
                                    precision    recall  f1-score   support

                           Refuted       0.83      0.82      0.82       305
                         Supported       0.65      0.84      0.74       122
               Not Enough Evidence       0.16      0.14      0.15        35
Conflicting Evidence/Cherrypicking       0.27      0.08      0.12        38

                          accuracy                           0.72       500
                         macro avg       0.48      0.47      0.46       500
                      weighted avg       0.70      0.72      0.70       500

Testing GPT4o Likert on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.85      0.82      0.83        92
                         Supported       0.62      0.92      0.74        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Ch

In [11]:
#new data
pipeline_results_gpt_likert = [mock_pipeline(d.datapoint) for d in dump_dev_test]

test_classifier(pipeline_results_gpt_likert, datapoints_dev_test, verbose=True)

y_pred (array([0, 1, 2, 3]), array([88, 55,  7,  2]))
y_true (array([0, 1, 2, 3]), array([92, 37, 11, 12]))
                                    precision    recall  f1-score   support

                           Refuted       0.85      0.82      0.83        92
                         Supported       0.62      0.92      0.74        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.72       152
                         macro avg       0.37      0.43      0.39       152
                      weighted avg       0.67      0.72      0.68       152



In [30]:
pipeline_results_gpt_likert = [mock_pipeline(d.datapoint) for d in dump_dev_test]

test_classifier(pipeline_results_gpt_likert, datapoints_dev_test, verbose=True)


y_pred (array([0, 1, 2, 3]), array([88, 55,  7,  2]))
y_true (array([0, 1, 2, 3]), array([92, 37, 11, 12]))
                                    precision    recall  f1-score   support

                           Refuted       0.85      0.82      0.83        92
                         Supported       0.62      0.92      0.74        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.72       152
                         macro avg       0.37      0.43      0.39       152
                      weighted avg       0.67      0.72      0.68       152



In [33]:
#whole dev
pipeline_results_gpt_likert = [mock_pipeline(d.datapoint) for d in dump]

test_classifier(pipeline_results_gpt_likert, datapoints, verbose=True)

y_pred (array([0, 1, 2, 3]), array([305, 143,  39,  13]))
y_true (array([0, 1, 2, 3]), array([305, 122,  35,  38]))
                                    precision    recall  f1-score   support

                           Refuted       0.81      0.81      0.81       305
                         Supported       0.61      0.71      0.66       122
               Not Enough Evidence       0.18      0.20      0.19        35
Conflicting Evidence/Cherrypicking       0.15      0.05      0.08        38

                          accuracy                           0.69       500
                         macro avg       0.44      0.44      0.43       500
                      weighted avg       0.67      0.69      0.67       500



In [9]:
#whole dev new data
pipeline_results_gpt_likert = [mock_pipeline(d.datapoint) for d in dump]

test_classifier(pipeline_results_gpt_likert, datapoints, verbose=True)

y_pred (array([0, 1, 2, 3]), array([299, 158,  32,  11]))
y_true (array([0, 1, 2, 3]), array([305, 122,  35,  38]))
                                    precision    recall  f1-score   support

                           Refuted       0.83      0.82      0.82       305
                         Supported       0.65      0.84      0.74       122
               Not Enough Evidence       0.16      0.14      0.15        35
Conflicting Evidence/Cherrypicking       0.27      0.08      0.12        38

                          accuracy                           0.72       500
                         macro avg       0.48      0.47      0.46       500
                      weighted avg       0.70      0.72      0.70       500



### Claude Likert scale based classifier

In [15]:
#ClaudeClassifier example
claude_likert_classifier = DefaultClassifier()

mock_pipeline = MockPipeline(
    classifier=claude_likert_classifier,
    dumps=DUMP_PATH_CLAUDE,
)

pipeline_result = mock_pipeline(dump_dev_test[0].datapoint)
pipeline_result.classification_result, str(pipeline_result.classification_result), str(
    pipeline_result.evidence_generation_result.evidences[0])

(ClassificationResult(probs=array([0.94791499, 0.01736167, 0.01736167, 0.01736167]), metadata=None),
 'Refuted',
 'Evidence(question=\'Was Netflix CEO Reed Hastings arrested?\', answer=\'No, Reed Hastings was not arrested. This claim is false.\', url=\'https://www.boomlive.in/world/posts-claiming-netflix-ceo-was-arrested-for-child-pornography-are-false-9919\', scraped_text=\'\\nPosts Claiming Netflix CEO Was Arrested For Child Pornography Are False Law enforcement officials said that there was no arrest and the rumour was published by a fringe news site, where it has since been removed. "The CEO of a Netflix – Reed Hastings – has been caught by FBI agents with 13,000 files of child pornography. He was arrested at his home in California," reads a Facebook post, archived here, shared more than 400 times since it was published on September 12, 2020. Screenshot taken on September 22, 2020, showing the false claim on Facebook The same claim also circulated on social media in Spanish, Portug

In [16]:
#final evaluation
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "Claude 3.5 Sonnet Likert")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing Claude 3.5 Sonnet Likert on dev
                                    precision    recall  f1-score   support

                           Refuted       0.87      0.70      0.78       305
                         Supported       0.74      0.66      0.70       122
               Not Enough Evidence       0.21      0.20      0.20        35
Conflicting Evidence/Cherrypicking       0.18      0.53      0.27        38

                          accuracy                           0.64       500
                         macro avg       0.50      0.52      0.49       500
                      weighted avg       0.74      0.64      0.68       500

Testing Claude 3.5 Sonnet Likert on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.87      0.63      0.73        92
                         Supported       0.74      0.70      0.72        37
               Not Enough Evidence       0.23      0.27      0.25        11

In [8]:
pipeline_results_claude_likert = [mock_pipeline(d.datapoint) for d in dump_dev_test]

test_classifier(pipeline_results_claude_likert, datapoints_dev_test, verbose=True)

y_pred (array([0, 1, 2, 3]), array([67, 35, 13, 37]))
y_true (array([0, 1, 2, 3]), array([92, 37, 11, 12]))
                                    precision    recall  f1-score   support

                           Refuted       0.87      0.63      0.73        92
                         Supported       0.74      0.70      0.72        37
               Not Enough Evidence       0.23      0.27      0.25        11
Conflicting Evidence/Cherrypicking       0.16      0.50      0.24        12

                          accuracy                           0.61       152
                         macro avg       0.50      0.53      0.49       152
                      weighted avg       0.73      0.61      0.65       152



In [9]:
#whole dev
pipeline_results_claude_likert = [mock_pipeline(d.datapoint) for d in dump]

test_classifier(pipeline_results_claude_likert, datapoints, verbose=True)

y_pred (array([0, 1, 2, 3]), array([247, 110,  34, 109]))
y_true (array([0, 1, 2, 3]), array([305, 122,  35,  38]))
                                    precision    recall  f1-score   support

                           Refuted       0.87      0.70      0.78       305
                         Supported       0.74      0.66      0.70       122
               Not Enough Evidence       0.21      0.20      0.20        35
Conflicting Evidence/Cherrypicking       0.18      0.53      0.27        38

                          accuracy                           0.64       500
                         macro avg       0.50      0.52      0.49       500
                      weighted avg       0.74      0.64      0.68       500



### Huggingface based classifier

In [19]:
#HuggingfaceClassifier example

#hf_classifier = HuggingfaceClassifier(model_path="/mnt/data/factcheck/averitec-data/pretrained_models/nli_4concat/deberta-v3-large/checkpoint-576")
hf_classifier = HuggingfaceClassifier(model_path="/mnt/data/factcheck/averitec-data/pretrained_models/nli_4concat/deberta-v3-large/checkpoint-576", rand_order_evidence=True, num_orders=10)

mock_pipeline = MockPipeline(
    classifier=hf_classifier,
    dumps=DUMP_PATH,
)

pipeline_result = mock_pipeline(dump_dev_test[0].datapoint)
pipeline_result.classification_result, str(pipeline_result.classification_result), str(
    pipeline_result.evidence_generation_result.evidences[0]
)

(ClassificationResult(probs=array([9.9823415e-01, 3.8673708e-04, 6.3261896e-04, 7.4644771e-04],
       dtype=float32), metadata={'logits': array([[ 5.6628795, -2.2362025, -1.6821896, -1.6098888],
        [ 5.627134 , -2.2523658, -1.7472888, -1.5814154],
        [ 5.662451 , -2.2239568, -1.6753788, -1.6271464],
        [ 5.644067 , -2.2748826, -1.7746158, -1.4930588],
        [ 5.642923 , -2.3031015, -1.7778584, -1.4794161],
        [ 5.6600285, -2.1879332, -1.6968951, -1.6178529],
        [ 5.563916 , -2.207106 , -1.7597977, -1.5954571],
        [ 5.66522  , -2.2148035, -1.7354256, -1.5842593],
        [ 5.558258 , -2.202819 , -1.7644497, -1.5769659],
        [ 5.571006 , -2.198929 , -1.766966 , -1.5608318]], dtype=float32), 'mean_logits': array([ 5.625788 , -2.2302098, -1.7380867, -1.5726292], dtype=float32)}),
 'Refuted',
 'Evidence(question=\'Was Reed Hastings, the CEO of Netflix, arrested by the FBI?\', answer=\'No, Reed Hastings was not arrested by the FBI.\', url=\'https://www.bo

In [18]:
#final evaluation
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "DeBERTa-v3-large")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing DeBERTa-v3-large on dev
                                    precision    recall  f1-score   support

                           Refuted       0.85      0.75      0.80       305
                         Supported       0.63      0.65      0.64       122
               Not Enough Evidence       0.00      0.00      0.00        35
Conflicting Evidence/Cherrypicking       0.10      0.26      0.14        38

                          accuracy                           0.63       500
                         macro avg       0.40      0.41      0.39       500
                      weighted avg       0.68      0.63      0.65       500

Testing DeBERTa-v3-large on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.86      0.76      0.81        92
                         Supported       0.59      0.62      0.61        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evi

In [20]:
#final evaluation (rand order)
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "DeBERTa-v3-large - rand order")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing DeBERTa-v3-large - rand order on dev
                                    precision    recall  f1-score   support

                           Refuted       0.87      0.74      0.80       305
                         Supported       0.67      0.70      0.69       122
               Not Enough Evidence       0.00      0.00      0.00        35
Conflicting Evidence/Cherrypicking       0.11      0.32      0.16        38

                          accuracy                           0.65       500
                         macro avg       0.41      0.44      0.41       500
                      weighted avg       0.70      0.65      0.67       500

Testing DeBERTa-v3-large - rand order on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.86      0.76      0.81        92
                         Supported       0.66      0.73      0.69        37
               Not Enough Evidence       0.00      0.00      0.00

In [13]:
#whole dev new gpt data
pipeline_results_hf = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_hf, datapoints, verbose=True)

y_pred (array([0, 1, 2, 3]), array([267, 125,   4, 104]))
y_true (array([0, 1, 2, 3]), array([305, 122,  35,  38]))
                                    precision    recall  f1-score   support

                           Refuted       0.85      0.75      0.80       305
                         Supported       0.63      0.65      0.64       122
               Not Enough Evidence       0.00      0.00      0.00        35
Conflicting Evidence/Cherrypicking       0.10      0.26      0.14        38

                          accuracy                           0.63       500
                         macro avg       0.40      0.41      0.39       500
                      weighted avg       0.68      0.63      0.65       500



In [17]:
#whole dev
pipeline_results_hf = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_hf, datapoints, verbose=True)

y_pred (array([0, 1, 2, 3]), array([265, 115,   5, 115]))
y_true (array([0, 1, 2, 3]), array([305, 122,  35,  38]))
                                    precision    recall  f1-score   support

                           Refuted       0.84      0.73      0.78       305
                         Supported       0.65      0.61      0.63       122
               Not Enough Evidence       0.40      0.06      0.10        35
Conflicting Evidence/Cherrypicking       0.10      0.29      0.14        38

                          accuracy                           0.62       500
                         macro avg       0.50      0.42      0.41       500
                      weighted avg       0.71      0.62      0.65       500



In [18]:
pipeline_results_hf = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_hf, datapoints_dev_test)


                                    precision    recall  f1-score   support

                           Refuted       0.86      0.77      0.81        92
                         Supported       0.60      0.57      0.58        37
               Not Enough Evidence       0.50      0.09      0.15        11
Conflicting Evidence/Cherrypicking       0.16      0.42      0.23        12

                          accuracy                           0.64       152
                         macro avg       0.53      0.46      0.44       152
                      weighted avg       0.71      0.64      0.66       152



In [14]:
#new gpt data
pipeline_results_hf = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_hf, datapoints_dev_test)


                                    precision    recall  f1-score   support

                           Refuted       0.86      0.76      0.81        92
                         Supported       0.59      0.62      0.61        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.06      0.17      0.09        12

                          accuracy                           0.62       152
                         macro avg       0.38      0.39      0.38       152
                      weighted avg       0.67      0.62      0.64       152



In [20]:
#with claude evidence
pipeline_results_hf = [mock_pipeline(d.datapoint) for d in dump_dev_test]

test_classifier(pipeline_results_hf, datapoints_dev_test)


                                    precision    recall  f1-score   support

                           Refuted       0.84      0.77      0.80        92
                         Supported       0.73      0.59      0.66        37
               Not Enough Evidence       0.67      0.36      0.47        11
Conflicting Evidence/Cherrypicking       0.13      0.33      0.19        12

                          accuracy                           0.66       152
                         macro avg       0.59      0.52      0.53       152
                      weighted avg       0.74      0.66      0.69       152



In [21]:
#with claude evidence whole dev
pipeline_results_hf = [mock_pipeline(d.datapoint) for d in dump]

test_classifier(pipeline_results_hf, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.83      0.76      0.80       305
                         Supported       0.77      0.59      0.67       122
               Not Enough Evidence       0.31      0.14      0.20        35
Conflicting Evidence/Cherrypicking       0.13      0.37      0.19        38

                          accuracy                           0.65       500
                         macro avg       0.51      0.47      0.46       500
                      weighted avg       0.72      0.65      0.68       500



In [14]:
pipeline_results_hf_rand = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_hf_rand, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.84      0.76      0.80        92
                         Supported       0.61      0.59      0.60        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.12      0.33      0.18        12

                          accuracy                           0.63       152
                         macro avg       0.39      0.42      0.40       152
                      weighted avg       0.67      0.63      0.65       152



In [17]:
#rand new data
pipeline_results_hf_rand = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_hf_rand, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.86      0.76      0.81        92
                         Supported       0.66      0.73      0.69        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.14      0.33      0.20        12

                          accuracy                           0.66       152
                         macro avg       0.42      0.46      0.42       152
                      weighted avg       0.69      0.66      0.67       152



In [16]:
#rand whole dev new data
pipeline_results_hf_rand = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_hf_rand, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.87      0.74      0.80       305
                         Supported       0.67      0.70      0.69       122
               Not Enough Evidence       0.00      0.00      0.00        35
Conflicting Evidence/Cherrypicking       0.11      0.32      0.16        38

                          accuracy                           0.65       500
                         macro avg       0.41      0.44      0.41       500
                      weighted avg       0.70      0.65      0.67       500



In [15]:
#rand whole dev
pipeline_results_hf_rand = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_hf_rand, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.84      0.73      0.78       305
                         Supported       0.66      0.60      0.63       122
               Not Enough Evidence       0.25      0.03      0.05        35
Conflicting Evidence/Cherrypicking       0.08      0.26      0.13        38

                          accuracy                           0.61       500
                         macro avg       0.46      0.41      0.40       500
                      weighted avg       0.69      0.61      0.64       500



In [11]:
#rand with Claude evidence
pipeline_results_hf_rand = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_hf_rand, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.85      0.77      0.81        92
                         Supported       0.73      0.59      0.66        37
               Not Enough Evidence       0.67      0.36      0.47        11
Conflicting Evidence/Cherrypicking       0.16      0.42      0.23        12

                          accuracy                           0.67       152
                         macro avg       0.60      0.54      0.54       152
                      weighted avg       0.75      0.67      0.70       152



In [12]:
#rand with Claude evidence - whole dev
pipeline_results_hf_rand = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_hf_rand, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.84      0.76      0.80       305
                         Supported       0.75      0.63      0.69       122
               Not Enough Evidence       0.29      0.14      0.19        35
Conflicting Evidence/Cherrypicking       0.14      0.39      0.21        38

                          accuracy                           0.66       500
                         macro avg       0.51      0.48      0.47       500
                      weighted avg       0.73      0.66      0.68       500



### Simple average ensemble classifier

In [33]:
# AverageEnsemble classifier example
hf_classifier = HuggingfaceClassifier(model_path="/mnt/data/factcheck/averitec-data/pretrained_models/nli_4concat/deberta-v3-large/checkpoint-576")
hf_classifier_order = HuggingfaceClassifier(model_path="/mnt/data/factcheck/averitec-data/pretrained_models/nli_4concat/deberta-v3-large/checkpoint-576", rand_order_evidence=True, num_orders=10)
gpt_likert_classifier = DefaultClassifier()
claude_likert_classifier = DefaultClassifier()
rf_classifier = RandomForestClassifier()

mock_pipeline = MockPipeline(
    classifier=AverageEnsembleClassifier([rf_classifier, hf_classifier, gpt_likert_classifier], [0.1, 0.3, 0.6]),
    dumps=DUMP_PATH,
)

pipeline_result = mock_pipeline(dump_dev_test[0].datapoint)
pipeline_result.classification_result, str(pipeline_result.classification_result), str(
    pipeline_result.evidence_generation_result.evidences[0]
)

/home/ullriher/venvs/averitec/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeClassifier from version 1.5.1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ullriher/venvs/averitec/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestClassifier from version 1.5.1 when using version 1.5.0. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/ullriher/venvs/averitec/lib/python3.10/site-packages/sklearn/base.py:376: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from v

(ClassificationResult(probs=array([0.94059992, 0.01911592, 0.02588037, 0.01440378]), metadata={'clf_probs': [array([0.7238869 , 0.08580357, 0.15263095, 0.03767857]), array([9.9820745e-01, 3.9521002e-04, 6.6759303e-04, 7.2973495e-04],
       dtype=float32), array([0.94791499, 0.01736167, 0.01736167, 0.01736167])], 'weights': [0.1, 0.3, 0.6]}),
 'Refuted',
 'Evidence(question=\'Was Netflix CEO Reed Hastings arrested?\', answer=\'No, Reed Hastings was not arrested. This claim is false.\', url=\'https://www.boomlive.in/world/posts-claiming-netflix-ceo-was-arrested-for-child-pornography-are-false-9919\', scraped_text=\'\\nPosts Claiming Netflix CEO Was Arrested For Child Pornography Are False Law enforcement officials said that there was no arrest and the rumour was published by a fringe news site, where it has since been removed. "The CEO of a Netflix – Reed Hastings – has been caught by FBI agents with 13,000 files of child pornography. He was arrested at his home in California," reads a 

In [37]:
#final eval hf+gpt likert
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "HF+GPT Likert 01 09")

Testing HF+GPT Likert 01 09 on dev
                                    precision    recall  f1-score   support

                           Refuted       0.85      0.82      0.83        92
                         Supported       0.62      0.92      0.74        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.72       152
                         macro avg       0.37      0.43      0.39       152
                      weighted avg       0.67      0.72      0.68       152



In [22]:
#final eval hf+gpt likert
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "HF+GPT Likert")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing HF+GPT Likert on dev
                                    precision    recall  f1-score   support

                           Refuted       0.85      0.80      0.83       305
                         Supported       0.65      0.78      0.71       122
               Not Enough Evidence       0.00      0.00      0.00        35
Conflicting Evidence/Cherrypicking       0.16      0.24      0.19        38

                          accuracy                           0.70       500
                         macro avg       0.41      0.45      0.43       500
                      weighted avg       0.69      0.70      0.69       500

Testing HF+GPT Likert on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.87      0.79      0.83        92
                         Supported       0.63      0.89      0.74        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/

In [24]:
#final eval hf+claude likert
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "HF+Claude Likert")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing HF+Claude Likert on dev
                                    precision    recall  f1-score   support

                           Refuted       0.84      0.79      0.81       305
                         Supported       0.77      0.65      0.71       122
               Not Enough Evidence       0.25      0.11      0.16        35
Conflicting Evidence/Cherrypicking       0.16      0.39      0.23        38

                          accuracy                           0.68       500
                         macro avg       0.50      0.49      0.47       500
                      weighted avg       0.73      0.68      0.69       500

Testing HF+Claude Likert on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.83      0.77      0.80        92
                         Supported       0.74      0.68      0.70        37
               Not Enough Evidence       0.38      0.27      0.32        11
Conflicting Evi

In [14]:
#rf + gpt new data
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.85      0.80      0.83        92
                         Supported       0.62      0.92      0.74        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.71       152
                         macro avg       0.37      0.43      0.39       152
                      weighted avg       0.67      0.71      0.68       152



In [15]:
#rf+gpt whole dev new data 01 09
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_avg_ensemble, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.84      0.80      0.82       305
                         Supported       0.66      0.83      0.73       122
               Not Enough Evidence       0.14      0.14      0.14        35
Conflicting Evidence/Cherrypicking       0.17      0.08      0.11        38

                          accuracy                           0.71       500
                         macro avg       0.45      0.46      0.45       500
                      weighted avg       0.69      0.71      0.70       500



In [23]:
#hf+gpt whole dev new data 0.36 0.64
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_avg_ensemble, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.85      0.81      0.83       305
                         Supported       0.68      0.82      0.74       122
               Not Enough Evidence       0.23      0.29      0.26        35
Conflicting Evidence/Cherrypicking       0.24      0.11      0.15        38

                          accuracy                           0.72       500
                         macro avg       0.50      0.51      0.49       500
                      weighted avg       0.72      0.72      0.71       500



In [19]:
#hf+gpt whole dev new data 05 05
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_avg_ensemble, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.84      0.78      0.81       305
                         Supported       0.68      0.78      0.73       122
               Not Enough Evidence       0.19      0.31      0.24        35
Conflicting Evidence/Cherrypicking       0.30      0.16      0.21        38

                          accuracy                           0.70       500
                         macro avg       0.50      0.51      0.50       500
                      weighted avg       0.72      0.70      0.70       500



In [22]:
#hf order + gpt new data
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.87      0.78      0.82        92
                         Supported       0.63      0.89      0.74        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.13      0.17      0.15        12

                          accuracy                           0.70       152
                         macro avg       0.41      0.46      0.43       152
                      weighted avg       0.69      0.70      0.69       152



In [19]:
#hf + gpt new data
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.87      0.79      0.83        92
                         Supported       0.63      0.89      0.74        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.14      0.17      0.15        12

                          accuracy                           0.71       152
                         macro avg       0.41      0.46      0.43       152
                      weighted avg       0.69      0.71      0.69       152



In [30]:
#hf + gpt
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.84      0.80      0.82        92
                         Supported       0.61      0.73      0.67        37
               Not Enough Evidence       0.50      0.09      0.15        11
Conflicting Evidence/Cherrypicking       0.11      0.17      0.13        12

                          accuracy                           0.68       152
                         macro avg       0.52      0.45      0.44       152
                      weighted avg       0.70      0.68      0.68       152



In [31]:
#hf+gpt whole dev new data 01 09
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_avg_ensemble, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.84      0.81      0.83       305
                         Supported       0.65      0.84      0.73       122
               Not Enough Evidence       0.16      0.14      0.15        35
Conflicting Evidence/Cherrypicking       0.19      0.08      0.11        38

                          accuracy                           0.72       500
                         macro avg       0.46      0.47      0.45       500
                      weighted avg       0.70      0.72      0.70       500



In [20]:
#hf+gpt whole dev new data
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_avg_ensemble, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.85      0.80      0.83       305
                         Supported       0.65      0.78      0.71       122
               Not Enough Evidence       0.00      0.00      0.00        35
Conflicting Evidence/Cherrypicking       0.16      0.24      0.19        38

                          accuracy                           0.70       500
                         macro avg       0.41      0.45      0.43       500
                      weighted avg       0.69      0.70      0.69       500



In [23]:
#hf order +gpt whole dev new data
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_avg_ensemble, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.87      0.79      0.83       305
                         Supported       0.66      0.80      0.72       122
               Not Enough Evidence       0.00      0.00      0.00        35
Conflicting Evidence/Cherrypicking       0.16      0.26      0.20        38

                          accuracy                           0.70       500
                         macro avg       0.42      0.46      0.44       500
                      weighted avg       0.70      0.70      0.70       500



In [28]:
#hf+gpt whole dev
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_avg_ensemble, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.83      0.79      0.81       305
                         Supported       0.65      0.69      0.67       122
               Not Enough Evidence       0.22      0.06      0.09        35
Conflicting Evidence/Cherrypicking       0.10      0.18      0.13        38

                          accuracy                           0.67       500
                         macro avg       0.45      0.43      0.42       500
                      weighted avg       0.69      0.67      0.67       500



In [50]:
#hf with Claude
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.86      0.78      0.82        92
                         Supported       0.66      0.68      0.67        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.18      0.42      0.25        12

                          accuracy                           0.67       152
                         macro avg       0.42      0.47      0.43       152
                      weighted avg       0.69      0.67      0.68       152



In [39]:
# gpt, claude, hf
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.85      0.79      0.82        92
                         Supported       0.67      0.70      0.68        37
               Not Enough Evidence       0.25      0.09      0.13        11
Conflicting Evidence/Cherrypicking       0.17      0.33      0.23        12

                          accuracy                           0.68       152
                         macro avg       0.48      0.48      0.47       152
                      weighted avg       0.71      0.68      0.69       152



In [23]:
# claude, hf - claude ev
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.83      0.77      0.80        92
                         Supported       0.74      0.68      0.70        37
               Not Enough Evidence       0.38      0.27      0.32        11
Conflicting Evidence/Cherrypicking       0.17      0.33      0.22        12

                          accuracy                           0.68       152
                         macro avg       0.53      0.51      0.51       152
                      weighted avg       0.72      0.68      0.69       152



In [24]:
# claude, hf - claude ev - whole dev
pipeline_results_avg_ensemble = [mock_pipeline(d.datapoint) for d in dump]
test_classifier(pipeline_results_avg_ensemble, datapoints)

                                    precision    recall  f1-score   support

                           Refuted       0.84      0.79      0.81       305
                         Supported       0.77      0.65      0.71       122
               Not Enough Evidence       0.25      0.11      0.16        35
Conflicting Evidence/Cherrypicking       0.16      0.39      0.23        38

                          accuracy                           0.68       500
                         macro avg       0.50      0.49      0.47       500
                      weighted avg       0.73      0.68      0.69       500



### Weighted average ensemble classifier

In [36]:
# AverageEnsemble classifier fiting example
hf_classifier = HuggingfaceClassifier(model_path="/mnt/data/factcheck/averitec-data/pretrained_models/nli_4concat/deberta-v3-large/checkpoint-576")
hf_classifier_order = HuggingfaceClassifier(model_path="/mnt/data/factcheck/averitec-data/pretrained_models/nli_4concat/deberta-v3-large/checkpoint-576", rand_order_evidence=True, num_orders=10)
gpt_likert_classifier = DefaultClassifier()
claude_likert_classifier = DefaultClassifier()
rf_classifier = RandomForestClassifier()

avg_ensemble_classifier = AverageEnsembleClassifier([hf_classifier, claude_likert_classifier])

# fit the weights TODO using only part of the data -> to be able to validate results! 
#prepared for univariate (used for 2 classifiers) and multivariate (used for 3+ classifiers) optimization and 2 metrics f1 and cross-entropy
avg_ensemble_classifier.fit_weights(datapoints_dev_train, evidence_generation_results_dev_train, retrieval_results_dev_train, metric="cross-entropy")

mock_pipeline = MockPipeline(
    classifier=avg_ensemble_classifier,
    dumps=DUMP_PATH_CLAUDE,
)

pipeline_result = mock_pipeline(dump_dev_test[0].datapoint)
pipeline_result.classification_result, str(pipeline_result.classification_result), pipeline_result.datapoint.label, str(
    pipeline_result.evidence_generation_result.evidences[0]
)

 message: Solution found.
 success: True
  status: 0
     fun: 0.9674282566690495
       x: 0.10484900291804425
     nit: 12
    nfev: 12


(ClassificationResult(probs=array([0.95310931, 0.01558077, 0.01568365, 0.01562629]), metadata={'clf_probs': [array([9.9745589e-01, 3.7628258e-04, 1.3574883e-03, 8.1042910e-04],
       dtype=float32), array([0.94791499, 0.01736167, 0.01736167, 0.01736167])], 'weights': array([0.104849, 0.895151])}),
 'Refuted',
 'Refuted',
 'Evidence(question=\'Was Netflix CEO Reed Hastings arrested?\', answer=\'No, Reed Hastings was not arrested. This claim is false.\', url=\'https://www.boomlive.in/world/posts-claiming-netflix-ceo-was-arrested-for-child-pornography-are-false-9919\', scraped_text=\'\\nPosts Claiming Netflix CEO Was Arrested For Child Pornography Are False Law enforcement officials said that there was no arrest and the rumour was published by a fringe news site, where it has since been removed. "The CEO of a Netflix – Reed Hastings – has been caught by FBI agents with 13,000 files of child pornography. He was arrested at his home in California," reads a Facebook post, archived here, sha

In [26]:
#final evaluation hf+gpt likert - weighted, accuracy [0.32476382, 0.67523618]
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "HF+GPT Likert - weighted, accuracy")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing HF+GPT Likert - weighted, accuracy on dev
                                    precision    recall  f1-score   support

                           Refuted       0.83      0.84      0.84       305
                         Supported       0.65      0.83      0.73       122
               Not Enough Evidence       0.11      0.06      0.08        35
Conflicting Evidence/Cherrypicking       0.20      0.11      0.14        38

                          accuracy                           0.73       500
                         macro avg       0.45      0.46      0.44       500
                      weighted avg       0.69      0.73      0.70       500

Testing HF+GPT Likert - weighted, accuracy on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.86      0.86      0.86        92
                         Supported       0.61      0.89      0.73        37
               Not Enough Evidence       0.00      0.00

In [28]:
#final evaluation hf+gpt likert - weighted, f1
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "HF+GPT Likert - weighted, f1")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing HF+GPT Likert - weighted, f1 on dev
                                    precision    recall  f1-score   support

                           Refuted       0.84      0.82      0.83       305
                         Supported       0.65      0.83      0.73       122
               Not Enough Evidence       0.17      0.14      0.15        35
Conflicting Evidence/Cherrypicking       0.21      0.11      0.14        38

                          accuracy                           0.72       500
                         macro avg       0.47      0.47      0.46       500
                      weighted avg       0.70      0.72      0.70       500

Testing HF+GPT Likert - weighted, f1 on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.85      0.82      0.83        92
                         Supported       0.61      0.89      0.73        37
               Not Enough Evidence       0.00      0.00      0.00  

In [30]:
#final evaluation hf+gpt likert - weighted, cross-entropy
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "HF+GPT Likert - weighted, cross-entropy")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing HF+GPT Likert - weighted, cross-entropy on dev
                                    precision    recall  f1-score   support

                           Refuted       0.84      0.81      0.83       305
                         Supported       0.65      0.84      0.73       122
               Not Enough Evidence       0.16      0.14      0.15        35
Conflicting Evidence/Cherrypicking       0.19      0.08      0.11        38

                          accuracy                           0.72       500
                         macro avg       0.46      0.47      0.45       500
                      weighted avg       0.70      0.72      0.70       500

Testing HF+GPT Likert - weighted, cross-entropy on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.85      0.82      0.83        92
                         Supported       0.62      0.92      0.74        37
               Not Enough Evidence       0.00

In [33]:
#final evaluation hf+claude likert - weighted, accuracy
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "HF+Claude Likert - weighted, accuracy")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing HF+Claude Likert - weighted, accuracy on dev
                                    precision    recall  f1-score   support

                           Refuted       0.84      0.78      0.81       305
                         Supported       0.78      0.66      0.71       122
               Not Enough Evidence       0.18      0.11      0.14        35
Conflicting Evidence/Cherrypicking       0.18      0.45      0.26        38

                          accuracy                           0.68       500
                         macro avg       0.50      0.50      0.48       500
                      weighted avg       0.73      0.68      0.70       500

Testing HF+Claude Likert - weighted, accuracy on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.83      0.74      0.78        92
                         Supported       0.79      0.70      0.74        37
               Not Enough Evidence       0.25    

In [35]:
#final evaluation hf+claude likert - weighted, f1
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "HF+Claude Likert - weighted, f1")
complete_results.append(result)
complete_results.append(result_dev_test)

Testing HF+Claude Likert - weighted, f1 on dev
                                    precision    recall  f1-score   support

                           Refuted       0.87      0.70      0.77       305
                         Supported       0.78      0.66      0.71       122
               Not Enough Evidence       0.19      0.17      0.18        35
Conflicting Evidence/Cherrypicking       0.19      0.61      0.29        38

                          accuracy                           0.64       500
                         macro avg       0.51      0.53      0.49       500
                      weighted avg       0.75      0.64      0.68       500

Testing HF+Claude Likert - weighted, f1 on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.87      0.63      0.73        92
                         Supported       0.79      0.70      0.74        37
               Not Enough Evidence       0.17      0.18      

In [37]:
#final evaluation hf+claude likert - weighted, cross-entropy
result, result_dev_test = test_pipelines(mock_pipeline, dump, dump_dev_test, "HF+Claude Likert - weighted, cross-entropy")
complete_results.append(result)

Testing HF+Claude Likert - weighted, cross-entropy on dev
                                    precision    recall  f1-score   support

                           Refuted       0.87      0.69      0.77       305
                         Supported       0.78      0.66      0.71       122
               Not Enough Evidence       0.19      0.17      0.18        35
Conflicting Evidence/Cherrypicking       0.20      0.63      0.30        38

                          accuracy                           0.64       500
                         macro avg       0.51      0.54      0.49       500
                      weighted avg       0.75      0.64      0.68       500

Testing HF+Claude Likert - weighted, cross-entropy on dev_test
                                    precision    recall  f1-score   support

                           Refuted       0.86      0.62      0.72        92
                         Supported       0.79      0.70      0.74        37
               Not Enough Evidence     

In [45]:
#cross entropy deberta with gpt
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.80      0.80      0.80        92
                         Supported       0.60      0.76      0.67        37
               Not Enough Evidence       0.11      0.09      0.10        11
Conflicting Evidence/Cherrypicking       0.25      0.08      0.12        12

                          accuracy                           0.68       152
                         macro avg       0.44      0.43      0.42       152
                      weighted avg       0.66      0.68      0.67       152



In [25]:
#f1 rf with GPT
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.86      0.80      0.83        92
                         Supported       0.62      0.92      0.74        37
               Not Enough Evidence       0.11      0.09      0.10        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.72       152
                         macro avg       0.40      0.45      0.42       152
                      weighted avg       0.68      0.72      0.69       152



In [43]:
#f1 deberta with GPT
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.80      0.84      0.82        92
                         Supported       0.59      0.73      0.65        37
               Not Enough Evidence       0.33      0.09      0.14        11
Conflicting Evidence/Cherrypicking       0.14      0.08      0.11        12

                          accuracy                           0.70       152
                         macro avg       0.47      0.44      0.43       152
                      weighted avg       0.66      0.70      0.67       152



In [29]:
#f1 deberta order with GPT - new data
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.85      0.82      0.83        92
                         Supported       0.62      0.92      0.74        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.72       152
                         macro avg       0.37      0.43      0.39       152
                      weighted avg       0.67      0.72      0.68       152



In [25]:
#f1 deberta with GPT - new data
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.85      0.82      0.83        92
                         Supported       0.61      0.89      0.73        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.71       152
                         macro avg       0.37      0.43      0.39       152
                      weighted avg       0.66      0.71      0.68       152



In [27]:
#ce deberta with GPT - new data
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.85      0.82      0.83        92
                         Supported       0.62      0.92      0.74        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.72       152
                         macro avg       0.37      0.43      0.39       152
                      weighted avg       0.67      0.72      0.68       152



In [47]:
#acc deberta with GPT
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.81      0.84      0.82        92
                         Supported       0.59      0.73      0.65        37
               Not Enough Evidence       0.33      0.09      0.14        11
Conflicting Evidence/Cherrypicking       0.12      0.08      0.10        12

                          accuracy                           0.70       152
                         macro avg       0.46      0.44      0.43       152
                      weighted avg       0.67      0.70      0.68       152



In [58]:
#ce gpt with Claude with hf - claude ev
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.80      0.77      0.78        92
                         Supported       0.63      0.73      0.68        37
               Not Enough Evidence       0.29      0.18      0.22        11
Conflicting Evidence/Cherrypicking       0.08      0.08      0.08        12

                          accuracy                           0.66       152
                         macro avg       0.45      0.44      0.44       152
                      weighted avg       0.66      0.66      0.66       152



In [39]:
#f1 Claude with hf - claude ev
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.86      0.62      0.72        92
                         Supported       0.79      0.70      0.74        37
               Not Enough Evidence       0.17      0.18      0.17        11
Conflicting Evidence/Cherrypicking       0.20      0.67      0.30        12

                          accuracy                           0.61       152
                         macro avg       0.50      0.54      0.49       152
                      weighted avg       0.74      0.61      0.65       152



In [37]:
#ce Claude with hf - claude ev
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.86      0.62      0.72        92
                         Supported       0.79      0.70      0.74        37
               Not Enough Evidence       0.17      0.18      0.17        11
Conflicting Evidence/Cherrypicking       0.20      0.67      0.30        12

                          accuracy                           0.61       152
                         macro avg       0.50      0.54      0.49       152
                      weighted avg       0.74      0.61      0.65       152



In [41]:
#acc Claude with hf - claude ev
pipeline_results_avg_w_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_avg_w_ensemble, datapoints_dev_test)

                                    precision    recall  f1-score   support

                           Refuted       0.86      0.62      0.72        92
                         Supported       0.79      0.70      0.74        37
               Not Enough Evidence       0.17      0.18      0.17        11
Conflicting Evidence/Cherrypicking       0.20      0.67      0.30        12

                          accuracy                           0.61       152
                         macro avg       0.50      0.54      0.49       152
                      weighted avg       0.74      0.61      0.65       152



### Logistic regression based stacking classifier

In [47]:
#LogRegEnsemble classifier example
hf_classifier = HuggingfaceClassifier(model_path="/mnt/data/factcheck/averitec-data/pretrained_models/nli_4concat/deberta-v3-large/checkpoint-576")
gpt_likert_classifier = DefaultClassifier(evidence_generation_results=evidence_generation_results)
claude_likert_classifier = DefaultClassifier(evidence_generation_results=evidence_generation_results_claude)

logreg_ensemble_classifier = LogRegEnsembleClassifier([claude_likert_classifier, gpt_likert_classifier])

#need all labels to present in the data
logreg_ensemble_classifier.fit(datapoints_dev_train, evidence_generation_results_dev_train, retrieval_results_dev_train)

mock_pipeline = MockPipeline(
    classifier=logreg_ensemble_classifier,
    dumps=DUMP_PATH_CLAUDE,
)

pipeline_result = mock_pipeline(dump_dev_test[0].datapoint)
pipeline_result.classification_result, str(pipeline_result.classification_result), str(
    pipeline_result.evidence_generation_result.evidences[0]
)

(ClassificationResult(probs=array([0.91284996, 0.03829509, 0.02086459, 0.02799035]), metadata={'clf_probs': [array([0.94791499, 0.01736167, 0.01736167, 0.01736167]), array([0.94791499, 0.01736167, 0.01736167, 0.01736167])]}),
 'Refuted',
 'Evidence(question=\'Was Netflix CEO Reed Hastings arrested?\', answer=\'No, Reed Hastings was not arrested. This claim is false.\', url=\'https://www.boomlive.in/world/posts-claiming-netflix-ceo-was-arrested-for-child-pornography-are-false-9919\', scraped_text=\'\\nPosts Claiming Netflix CEO Was Arrested For Child Pornography Are False Law enforcement officials said that there was no arrest and the rumour was published by a fringe news site, where it has since been removed. "The CEO of a Netflix – Reed Hastings – has been caught by FBI agents with 13,000 files of child pornography. He was arrested at his home in California," reads a Facebook post, archived here, shared more than 400 times since it was published on September 12, 2020. Screenshot taken

In [39]:
pipeline_results_logreg_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_logreg_ensemble, datapoints_dev_test, verbose=True)

y_pred (array([0, 1]), array([108,  44]))
y_true (array([0, 1, 2, 3]), array([92, 37, 11, 12]))
                                    precision    recall  f1-score   support

                           Refuted       0.79      0.92      0.85        92
                         Supported       0.61      0.73      0.67        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.74       152
                         macro avg       0.35      0.41      0.38       152
                      weighted avg       0.63      0.74      0.68       152



/home/mlynatom/aic_averitec/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mlynatom/aic_averitec/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mlynatom/aic_averitec/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [22]:
#hf with Claude
pipeline_results_logreg_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_logreg_ensemble, datapoints_dev_test, verbose=True)

y_pred (array([0, 1]), array([120,  32]))
y_true (array([0, 1, 2, 3]), array([92, 37, 11, 12]))
                                    precision    recall  f1-score   support

                           Refuted       0.73      0.96      0.83        92
                         Supported       0.78      0.68      0.72        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.74       152
                         macro avg       0.38      0.41      0.39       152
                      weighted avg       0.63      0.74      0.68       152



/home/mlynatom/aic_averitec/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mlynatom/aic_averitec/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mlynatom/aic_averitec/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

In [48]:
#gpt with Claude
pipeline_results_logreg_ensemble = [mock_pipeline(d.datapoint) for d in dump_dev_test]
test_classifier(pipeline_results_logreg_ensemble, datapoints_dev_test, verbose=True)

y_pred (array([0, 1]), array([108,  44]))
y_true (array([0, 1, 2, 3]), array([92, 37, 11, 12]))
                                    precision    recall  f1-score   support

                           Refuted       0.77      0.90      0.83        92
                         Supported       0.64      0.76      0.69        37
               Not Enough Evidence       0.00      0.00      0.00        11
Conflicting Evidence/Cherrypicking       0.00      0.00      0.00        12

                          accuracy                           0.73       152
                         macro avg       0.35      0.41      0.38       152
                      weighted avg       0.62      0.73      0.67       152



/home/mlynatom/aic_averitec/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mlynatom/aic_averitec/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/home/mlynatom/aic_averitec/venvs/averitec/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modif

### Process complete results

In [55]:
import pickle 

pickle.dump(complete_results, open("/mnt/data/factcheck/averitec-data/data_store/complete_results.pkl", "wb"))

In [54]:
complete_results

[{'Refuted': {'precision': 0.8327759197324415,
   'recall': 0.8163934426229508,
   'f1-score': 0.8245033112582781,
   'support': 305.0},
  'Supported': {'precision': 0.6518987341772152,
   'recall': 0.8442622950819673,
   'f1-score': 0.7357142857142858,
   'support': 122.0},
  'Not Enough Evidence': {'precision': 0.15625,
   'recall': 0.14285714285714285,
   'f1-score': 0.14925373134328357,
   'support': 35.0},
  'Conflicting Evidence/Cherrypicking': {'precision': 0.2727272727272727,
   'recall': 0.07894736842105263,
   'f1-score': 0.12244897959183673,
   'support': 38.0},
  'accuracy': 0.72,
  'macro avg': {'precision': 0.47841298165923235,
   'recall': 0.47061506224577837,
   'f1-score': 0.45798007697692106,
   'support': 500.0},
  'weighted avg': {'precision': 0.6987213749033026,
   'recall': 0.72,
   'f1-score': 0.7022151892248448,
   'support': 500.0},
  'cls_name': 'GPT4o Likert - dev'},
 {'Refuted': {'precision': 0.8522727272727273,
   'recall': 0.8152173913043478,
   'f1-score'

In [45]:
#process complete results to a pandas dataframe and export as latex table - use macro average f1, recall, precision and accuracy
import pandas as pd

df = pd.DataFrame(complete_results)
df = df.drop(columns=["Refuted", "Supported", "Not Enough Evidence", "Conflicting Evidence/Cherrypicking"])
#process macro avg column to multiple columns and name the with macro_avg_<metric>
df = pd.concat([df, df["macro avg"].apply(pd.Series)], axis=1)
df = df.drop(columns=["macro avg"])
df = df.rename(columns={"f1-score": "macro_avg_f1", "recall": "macro_avg_recall", "precision": "macro_avg_precision", "support": "macro_avg_support"})
df.drop(columns=["macro_avg_support", "weighted avg"], inplace=True)
df

,accuracy,cls_name,macro_avg_precision,macro_avg_recall,macro_avg_f1
0,0.720000,GPT4o Likert - dev,0.478413,0.470615,0.457980
1,0.717105,GPT4o Likert - dev_test,0.367614,0.433534,0.393116
2,0.644000,Claude 3.5 Sonnet Likert - dev,0.498032,0.522972,0.487161
3,0.611842,Claude 3.5 Sonnet Likert - dev_test,0.500365,0.526466,0.486670
4,0.634000,DeBERTa-v3-large - dev,0.395522,0.414560,0.394431
5,0.625000,DeBERTa-v3-large - dev_test,0.379614,0.387289,0.376884
6,0.648000,DeBERTa-v3-large - rand order - dev,0.412777,0.439193,0.412364
7,0.664474,DeBERTa-v3-large - rand order - dev_test,0.415166,0.455983,0.424170
8,0.698000,HF+GPT Likert - dev,0.413772,0.454702,0.430381
9,0.710526,HF+GPT Likert - dev_test,0.411630,0.463009,0.431241


In [51]:
#change order of columns
df = df[["cls_name", "accuracy", "macro_avg_f1", "macro_avg_precision", "macro_avg_recall"]]

In [53]:
print(df.to_latex(float_format="%.2f", index=False, header=["Classifier", "Accuracy", "Macro F1", "Macro Precision", "Macro Recall"]))

\begin{tabular}{lrrrr}
\toprule
Classifier & Accuracy & Macro F1 & Macro Precision & Macro Recall \\
\midrule
GPT4o Likert - dev & 0.72 & 0.46 & 0.48 & 0.47 \\
GPT4o Likert - dev_test & 0.72 & 0.39 & 0.37 & 0.43 \\
Claude 3.5 Sonnet Likert - dev & 0.64 & 0.49 & 0.50 & 0.52 \\
Claude 3.5 Sonnet Likert - dev_test & 0.61 & 0.49 & 0.50 & 0.53 \\
DeBERTa-v3-large - dev & 0.63 & 0.39 & 0.40 & 0.41 \\
DeBERTa-v3-large - dev_test & 0.62 & 0.38 & 0.38 & 0.39 \\
DeBERTa-v3-large - rand order - dev & 0.65 & 0.41 & 0.41 & 0.44 \\
DeBERTa-v3-large - rand order - dev_test & 0.66 & 0.42 & 0.42 & 0.46 \\
HF+GPT Likert - dev & 0.70 & 0.43 & 0.41 & 0.45 \\
HF+GPT Likert - dev_test & 0.71 & 0.43 & 0.41 & 0.46 \\
HF+Claude Likert - dev & 0.68 & 0.47 & 0.50 & 0.49 \\
HF+Claude Likert - dev_test & 0.68 & 0.51 & 0.53 & 0.51 \\
HF+GPT Likert - weighted, accuracy - dev & 0.73 & 0.44 & 0.45 & 0.46 \\
HF+GPT Likert - weighted, accuracy - dev_test & 0.74 & 0.40 & 0.37 & 0.44 \\
HF+GPT Likert - weighted, f1 - dev 

## Full Eval of the Best Classifier

### Just Likert GPT

In [45]:


classifier = DefaultClassifier()

pipeline = MockPipeline(classifier=classifier, dumps=DUMP_PATH)

with open("/mnt/data/factcheck/averitec-data/data/dev.json") as f:
    dataset = json.load(f)
    seq = 0
    for d in dataset:
        d["claim_id"] = seq
        seq += 1
    
datapoints = [Datapoint.from_dict(d) for d in dataset]
submission = []
dump = []

for dp in tqdm(datapoints):
    pipeline_result = pipeline(dp)
    submission.append(pipeline_result.to_submission())
    dump.append(pipeline_result)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/submission_dev_gpt_likert.json", "w") as f:
        json.dump(submission, f, indent=4)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/pipeline_result_dev_gpt_likert.pkl", "wb") as f:
        pickle.dump(dump, f)

[0.37138793 0.62861207]


100%|██████████| 500/500 [00:52<00:00,  9.50it/s]


In [46]:
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submission_dev_gpt_likert.json

Question-only score (HU-meteor):             0.44559202247101487
Question-answer score (HU-meteor):           0.27486491800924373
Veracity F1 scores:
 * Supported:                                0.6566037735849056
 * Refuted:                                  0.8098360655737705
 * Not Enough Evidence:                      0.1891891891891892
 * Conflicting Evidence/Cherrypicking:       0.0784313725490196
 * macro:                                    0.43351510022422124
 * acc:                                      0.686
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.672
 * Veracity scores (meteor @ 0.2):           0.532
 * Veracity scores (meteor @ 0.25):          0.394
 * Veracity scores (meteor @ 0.3):           0.256
 * Veracity scores (meteor @ 0.4):           0.106
 * Veracity scores (meteor @ 0.5):           0.03
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.18721511530819013
 * Veracity score

### Just Claude

In [24]:
classifier = DefaultClassifier()
pipeline = MockPipeline(classifier=classifier, dumps=DUMP_PATH_CLAUDE)

with open("/mnt/data/factcheck/averitec-data/data/dev.json") as f:
    dataset = json.load(f)
    seq = 0
    for d in dataset:
        d["claim_id"] = seq
        seq += 1
    
datapoints = [Datapoint.from_dict(d) for d in dataset]
submission = []
dump = []

for dp in tqdm(datapoints):
    pipeline_result = pipeline(dp)
    submission.append(pipeline_result.to_submission())
    dump.append(pipeline_result)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/submission_dev_claude_likert.json", "w") as f:
        json.dump(submission, f, indent=4)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/pipeline_result_dev_claude_likert.pkl", "wb") as f:
        pickle.dump(dump, f)

100%|██████████| 500/500 [00:42<00:00, 11.68it/s]


In [25]:
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submission_dev_claude_likert.json

Question-only score (HU-meteor):             0.43356239977995653
Question-answer score (HU-meteor):           0.2756433945181975
Veracity F1 scores:
 * Supported:                                0.6982758620689655
 * Refuted:                                  0.7753623188405797
 * Not Enough Evidence:                      0.2028985507246377
 * Conflicting Evidence/Cherrypicking:       0.272108843537415
 * macro:                                    0.4871613937928995
 * acc:                                      0.644
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.63
 * Veracity scores (meteor @ 0.2):           0.496
 * Veracity scores (meteor @ 0.25):          0.35
 * Veracity scores (meteor @ 0.3):           0.234
 * Veracity scores (meteor @ 0.4):           0.094
 * Veracity scores (meteor @ 0.5):           0.038
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.18399764934948373
 * Veracity scores (P

### Deberta only

In [63]:
from tqdm import tqdm
from classification import HuggingfaceClassifier
classifier = HuggingfaceClassifier(model_path="/mnt/data/factcheck/averitec-data/pretrained_models/nli_4concat/deberta-v3-large/checkpoint-576")

pipeline = MockPipeline(classifier=classifier, dumps=DUMP_PATH_CLAUDE)

with open("/mnt/data/factcheck/averitec-data/data/dev.json") as f:
    dataset = json.load(f)
    seq = 0
    for d in dataset:
        d["claim_id"] = seq
        seq += 1
    
datapoints = [Datapoint.from_dict(d) for d in dataset]
submission = []
dump = []

for dp in tqdm(datapoints):
    pipeline_result = pipeline(dp)
    submission.append(pipeline_result.to_submission())
    dump.append(pipeline_result)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/submission_dev_hf_dc.json", "w") as f:
        json.dump(submission, f, indent=4)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/pipeline_result_dev_hf_dc.pkl", "wb") as f:
        pickle.dump(dump, f)

100%|██████████| 500/500 [01:08<00:00,  7.32it/s]


In [48]:
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submission_dev_hf.json

Question-only score (HU-meteor):             0.44559202247101487
Question-answer score (HU-meteor):           0.27486491800924373
Veracity F1 scores:
 * Supported:                                0.6329113924050633
 * Refuted:                                  0.7824561403508772
 * Not Enough Evidence:                      0.1
 * Conflicting Evidence/Cherrypicking:       0.1437908496732026
 * macro:                                    0.4147895956072858
 * acc:                                      0.622
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.606
 * Veracity scores (meteor @ 0.2):           0.474
 * Veracity scores (meteor @ 0.25):          0.358
 * Veracity scores (meteor @ 0.3):           0.232
 * Veracity scores (meteor @ 0.4):           0.098
 * Veracity scores (meteor @ 0.5):           0.028
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.18721511530819013
 * Veracity scores (Position Sta

In [64]:
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submission_dev_hf_dc.json

Question-only score (HU-meteor):             0.43356239977995653
Question-answer score (HU-meteor):           0.2756433945181975
Veracity F1 scores:
 * Supported:                                0.6666666666666666
 * Refuted:                                  0.7952218430034129
 * Not Enough Evidence:                      0.19607843137254902
 * Conflicting Evidence/Cherrypicking:       0.19047619047619047
 * macro:                                    0.46211078287970475
 * acc:                                      0.648
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.634
 * Veracity scores (meteor @ 0.2):           0.482
 * Veracity scores (meteor @ 0.25):          0.334
 * Veracity scores (meteor @ 0.3):           0.222
 * Veracity scores (meteor @ 0.4):           0.088
 * Veracity scores (meteor @ 0.5):           0.03
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.18399764934948373
 * Veracity scor

### Average Ensemble

In [65]:
hf_classifier = HuggingfaceClassifier(model_path="/mnt/data/factcheck/averitec-data/pretrained_models/nli_4concat/deberta-v3-large/checkpoint-576")
claude_likert_classifier = DefaultClassifier(evidence_generation_results=evidence_generation_results_claude)

pipeline = MockPipeline(
    classifier=AverageEnsembleClassifier([hf_classifier, claude_likert_classifier]),
    dumps=DUMP_PATH_CLAUDE,
)

with open("/mnt/data/factcheck/averitec-data/data/dev.json") as f:
    dataset = json.load(f)
    seq = 0
    for d in dataset:
        d["claim_id"] = seq
        seq += 1
    
datapoints = [Datapoint.from_dict(d) for d in dataset]
submission = []
dump = []

for dp in tqdm(datapoints):
    pipeline_result = pipeline(dp)
    submission.append(pipeline_result.to_submission())
    dump.append(pipeline_result)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/submission_dev_avg_hf_claude_dc.json", "w") as f:
        json.dump(submission, f, indent=4)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/pipeline_result_dev_avg_hf_claude_dc.pkl", "wb") as f:
        pickle.dump(dump, f)

[0.76393648 0.23606352]


100%|██████████| 500/500 [01:08<00:00,  7.34it/s]


In [66]:
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submission_dev_avg_hf_claude_dc.json

Question-only score (HU-meteor):             0.43356239977995653
Question-answer score (HU-meteor):           0.2756433945181975
Veracity F1 scores:
 * Supported:                                0.7053571428571429
 * Refuted:                                  0.8108108108108109
 * Not Enough Evidence:                      0.1568627450980392
 * Conflicting Evidence/Cherrypicking:       0.22556390977443608
 * macro:                                    0.4746486521351072
 * acc:                                      0.676
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.662
 * Veracity scores (meteor @ 0.2):           0.512
 * Veracity scores (meteor @ 0.25):          0.356
 * Veracity scores (meteor @ 0.3):           0.238
 * Veracity scores (meteor @ 0.4):           0.094
 * Veracity scores (meteor @ 0.5):           0.034
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.18399764934948373
 * Veracity score

### Weighted Average Ensemble

In [43]:
from tqdm import tqdm

pipeline = MockPipeline(classifier=avg_ensemble_classifier, dumps=DUMP_PATH)
print(avg_ensemble_classifier.weights)

with open("/mnt/data/factcheck/averitec-data/data/dev.json") as f:
    dataset = json.load(f)
    seq = 0
    for d in dataset:
        d["claim_id"] = seq
        seq += 1
    
datapoints = [Datapoint.from_dict(d) for d in dataset]
submission = []
dump = []

for dp in tqdm(datapoints):
    pipeline_result = pipeline(dp)
    submission.append(pipeline_result.to_submission())
    dump.append(pipeline_result)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/submission_dev_avg_clf.json", "w") as f:
        json.dump(submission, f, indent=4)
    with open(f"/mnt/data/factcheck/averitec-data/data_store/pipeline_result_dev_avg_clf.pkl", "wb") as f:
        pickle.dump(dump, f)

[0.37138793 0.62861207]


100%|██████████| 500/500 [01:15<00:00,  6.61it/s]


In [44]:
%run src/prediction/evaluate_veracity.py --label_file /mnt/data/factcheck/averitec-data/data/dev.json --prediction_file /mnt/data/factcheck/averitec-data/data_store/submission_dev_avg_clf.json

Question-only score (HU-meteor):             0.44559202247101487
Question-answer score (HU-meteor):           0.27486491800924373
Veracity F1 scores:
 * Supported:                                0.6666666666666666
 * Refuted:                                  0.8181818181818182
 * Not Enough Evidence:                      0.1509433962264151
 * Conflicting Evidence/Cherrypicking:       0.136986301369863
 * macro:                                    0.44319454561119076
 * acc:                                      0.694
--------------------
AVeriTeC scores:
 * Veracity scores (meteor @ 0.1):           0.678
 * Veracity scores (meteor @ 0.2):           0.528
 * Veracity scores (meteor @ 0.25):          0.392
 * Veracity scores (meteor @ 0.3):           0.252
 * Veracity scores (meteor @ 0.4):           0.104
 * Veracity scores (meteor @ 0.5):           0.032
--------------------
AVeriTeC scores by type @ 0.25:
 * Veracity scores (Event/Property Claim):   0.18721511530819013
 * Veracity score